In [36]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np

In [37]:
df = pd.read_excel('data/Il_1522_dati_provinciali_settimanali_2013_III_trim_2024.xlsx', sheet_name='tav.2')

In [38]:
# Initialize lists to store the processed data
processed_data = []

# Get the column names
columns = df.columns

# Current year being processed
current_year = None

In [39]:
# Iterate through the rows
for idx, row in df.iterrows():
    # Check if this row contains a year
    if pd.notna(row['Unnamed: 0']) and str(row['Unnamed: 0']).strip().isdigit():
        current_year = int(row['Unnamed: 0'])
        continue
        
    # Skip rows without province name or "Numero Settimana" row
    if pd.isna(row['Unnamed: 0']) or row['Unnamed: 0'] == 'Numero Settimana' or str(row['Unnamed: 0']).startswith('Tavola'):
        continue
        
    if current_year and pd.notna(row['Unnamed: 0']):
        province = row['Unnamed: 0']
        
        # Process each week
        for week_num in range(53):  # Assuming maximum 53 weeks in a year
            col_name = f'Unnamed: {week_num + 1}'
            if col_name in columns and pd.notna(row[col_name]):
                # Calculate the date for the end of the week
                try:
                    date = datetime(current_year, 1, 1) + timedelta(weeks=week_num, days=6)
                    calls = float(str(row[col_name]).replace(',', '.'))
                    
                    if calls > 0:  # Only add rows with actual calls
                        processed_data.append({
                            'date': date.strftime('%Y-%m-%d'),
                            'province': province,
                            'calls': calls
                        })
                except (ValueError, TypeError):
                    continue

# Create DataFrame from processed data
result_df = pd.DataFrame(processed_data)

# Sort by date and province
result_df = result_df.sort_values(['date', 'province'])

result_df = result_df.drop_duplicates(subset=['date', 'province', 'calls'])


In [40]:
result_df.to_csv('output/calls.csv', index=False, encoding='UTF-8')